In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
import wandb

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
X_train, Y_train, X_test, Y_test = torch.load('./data/dataset_2_40D.pt')

# 将数据移动到适当的设备
X_train = X_train.to(device)
Y_train = Y_train.to(device)
X_test = X_test.to(device)
Y_test = Y_test.to(device)

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

(torch.Size([8000, 41]),
 torch.Size([8000]),
 torch.Size([2000, 41]),
 torch.Size([2000]))

In [4]:
# 使用 DataLoader 进行批处理
l = 64
train_dataset = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=l, shuffle=True)

# 打印第一个批次的大小
for x, y in train_loader:
    print(x.shape, y.shape)
    break

torch.Size([64, 41]) torch.Size([64])


In [5]:
D = 40
m = 100

In [6]:
class Model(nn.Module):
    def __init__(self, input, hidden_layer, output):
        super(Model, self).__init__()
        self.relu = nn.ReLU()
        self.hidden_dim = hidden_layer
        self.W = nn.Parameter(torch.rand(input, hidden_layer, device=device), requires_grad=True)
        # HE初始化
        nn.init.kaiming_normal_(self.W, mode='fan_in', nonlinearity='relu')
        self.a = nn.Parameter(torch.rand(hidden_layer, output, device=device), requires_grad=True)
        nn.init.kaiming_normal_(self.a, mode='fan_in', nonlinearity='relu')
        
    def forward(self, x):
        # print(x.shape)
        z1 = self.relu(torch.mm(x, self.W))
        # print(z1.shape)
        z2 = torch.mm(z1, self.a) / self.hidden_dim
        return z2

    def loss(self, y_pred, y_true):
        return (y_pred - y_true.reshape(y_pred.shape)) ** 2

model = Model(D + 1, m, 1).to(device)

# 计算模型W和a的Norm
def get_norm(model):
    return torch.norm(model.W).item(), torch.norm(model.a).item()

get_norm(model)

(8.94745922088623, 13.344766616821289)

In [7]:
epochs = 10000
lr = 1
C = 1
_lambda = 4
r = 0
epsilon = 1e-8
beta_1 = 0.9
beta_2 = 0.999
m_a, m_w = 0, 0
v = 0

train_losses = []
test_losses = []

In [8]:
config = {
    'learning_rate': lr,
    'batch_size': l,
    'epochs': epochs,
    'hidden_layer': m,
    'input': D + 1,
    'output': 1,
    'optimizer': 'Adam_Restart_SAV',
    'Approx Method': 'PM',
    'C': C,
    '_lambda': _lambda,
    'r': r,
    'epsilon': epsilon,
    
}

In [9]:
import datetime

date = datetime.datetime.now().strftime("%m%d%H%M")
wandb.init(project='Numerical Method', name=f"PM_A_ReSAV_Example_2_{date}", config=config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pheonizard (pheonizard-university-of-nottingham). Use `wandb login --relogin` to force relogin


In [10]:
for epoch in range(epochs):
    m_a, m_w = 0, 0
    v = 0
    cnt = 0
    for X, Y in train_loader:
        loss = model.loss(model(X), Y).sum()
        r = torch.sqrt(torch.tensor(loss + C, device=device))
        loss.backward()
        with torch.no_grad():
            #=========Nonlinear Term==========
            N_a_init = model.a.grad
            N_w_init = model.W.grad
            m_a = beta_1 * m_a + (1 - beta_1) * N_a_init
            m_w = beta_1 * m_w + (1 - beta_1) * N_w_init
            
            v = beta_2 * v + (1 - beta_2) * (torch.norm(N_a_init) ** 2 + torch.norm(N_w_init) ** 2)
            m_a_hat = m_a / (1 - beta_1 ** (cnt + 1))
            m_w_hat = m_w / (1 - beta_1 ** (cnt + 1))

            v_hat = v / (1 - beta_2 ** (cnt + 1))
            N_a = m_a_hat
            N_w = m_w_hat
            #=========Time Step Update========
            adaptive_lr = lr / (torch.sqrt(v_hat) + epsilon)
            #=========SAV Update========== 
            theta_a_2 = adaptive_lr * N_a / (torch.sqrt(loss + C) * (1 + adaptive_lr * _lambda))
            theta_w_2 = adaptive_lr * N_w / (torch.sqrt(loss + C) * (1 + adaptive_lr * _lambda))
            r = r / (1 + adaptive_lr * (torch.sum(N_a * (N_a / (1 + adaptive_lr * _lambda))) + torch.sum(N_w * (N_w) / (1 + adaptive_lr * _lambda))) / (2 * (loss + C)))
            model.a -= r.item() * theta_a_2
            model.W -= r.item() * theta_w_2
            model.a.grad.zero_()
            model.W.grad.zero_()
            cnt += 1
    with torch.no_grad():
        train_loss = model.loss(model(X_train), Y_train).mean()
        test_loss = model.loss(model(X_test), Y_test).mean()
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        norm = get_norm(model)
        wandb.log({'epoch': epoch + 1,
                   'train_loss': train_loss, 
                   'test_loss': test_loss,
                   'norm_W': norm[0],
                   'norm_a': norm[1],
                   'accuracy': 1 - test_loss,
                   'r': r.item(),
                   'adaptive_lr': adaptive_lr.item()})
        print(f'epoch {epoch + 1}, loss {train_loss:.6f}, test loss {test_loss:.6f}')

/tmp/ipykernel_13577/1435131760.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  r = torch.sqrt(torch.tensor(loss + C, device=device))


epoch 1, loss 0.060264, test loss 0.064435
epoch 2, loss 0.050941, test loss 0.056205
epoch 3, loss 0.041550, test loss 0.048672
epoch 4, loss 0.031767, test loss 0.038812
epoch 5, loss 0.023792, test loss 0.030701
epoch 6, loss 0.019109, test loss 0.025618
epoch 7, loss 0.014691, test loss 0.020768
epoch 8, loss 0.011748, test loss 0.017612
epoch 9, loss 0.009300, test loss 0.014885
epoch 10, loss 0.008048, test loss 0.013117
epoch 11, loss 0.007012, test loss 0.011946
epoch 12, loss 0.006167, test loss 0.011221
epoch 13, loss 0.005374, test loss 0.010232
epoch 14, loss 0.005002, test loss 0.009883
epoch 15, loss 0.004878, test loss 0.010111
epoch 16, loss 0.004371, test loss 0.009244
epoch 17, loss 0.004061, test loss 0.009014
epoch 18, loss 0.003778, test loss 0.008784
epoch 19, loss 0.003637, test loss 0.008598
epoch 20, loss 0.003451, test loss 0.008547
epoch 21, loss 0.003609, test loss 0.008798
epoch 22, loss 0.003295, test loss 0.008286
epoch 23, loss 0.003326, test loss 0.0084

In [1]:
wandb.finish()

NameError: name 'wandb' is not defined